<h3> Importation des librairies et sélection des options d'affichage </h3>

In [6]:
import networkx as nx
import gurobipy
from gurobipy import Model, GRB
import matplotlib.pyplot as plt
import sys
import random
import csv

options = {
    "font_size": 20,
    "node_size": 1000,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 5,
    "width": 5,
    "with_labels": True,
}

Fonction de géneration d'instance de graphique avec sélection de type de graphe

In [7]:
# Generer des instances de graphes

def generate_graph(graph_type, n_nodes, p=None):
    """
    Génère un graphe selon le type spécifié.
    :param graph_type: Type de graphe ('complete', 'cycle', 'random').
    :param n_nodes: Nombre de nœuds.
    :param p: Probabilité pour les graphes aléatoires (nécessaire pour 'random').
    :return: Graphe généré.
    """
    if graph_type == 'complete':
        G = nx.complete_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'cycle':
        G = nx.cycle_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'random':
        G = nx.erdos_renyi_graph(n_nodes, p, directed=True)
    else:
        raise ValueError("Type de graphe non supporté")
    return G


Création des valeurs de capacité et de coût 

In [8]:
def assign_capacities_and_costs(G, capacity_range, cost_range):
    """
    Assigne des capacités et des coûts de routage aux arcs.
    :param G: Graphe.
    :param capacity_range: Intervalle pour les capacités (e.g., (1, 10)).
    :param cost_range: Intervalle pour les coûts (e.g., (1, 5)).
    :return: Graphe avec attributs assignés.
    """
    for u, v in G.edges:
        G[u][v]['capacity'] = random.randint(*capacity_range)
        G[u][v]['cost'] = random.uniform(*cost_range)
    return G

Generation de la demande du traffic 

In [9]:
def generate_traffic_demand(G, S_size, demand_range):
    """
    Génère des sous-ensembles S de nœuds et des demandes de trafic.
    :param G: Graphe.
    :param S_size: Taille de l'ensemble S.
    :param demand_range: Intervalle pour les demandes de trafic (e.g., (1, 20)).
    :return: Ensemble S et vecteur des demandes de trafic.
    """
    nodes = list(G.nodes)
    S = random.sample(nodes, S_size)
    traffic_demand = {}
    for i in S:
        for j in S:
            if i != j:
                traffic_demand[(i, j)] = random.randint(*demand_range)
    return S, traffic_demand

Sauvegarde de l'instance dans un csv

In [10]:
def save_to_csv(G, S, traffic_demand, filename):
    """
    Sauvegarde le graphe, les capacités, les coûts et les demandes de trafic dans un fichier CSV.
    :param G: Graphe.
    :param S: Sous-ensemble des nœuds.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param filename: Nom du fichier CSV.
    """
    with open("graph/"+filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Écriture des arcs avec capacités et coûts
        writer.writerow(['Source', 'Target', 'Capacity', 'Cost'])
        for u, v in G.edges:
            writer.writerow([u, v, G[u][v]['capacity'], G[u][v]['cost']])
        
        # Écriture des demandes de trafic
        writer.writerow([])
        writer.writerow(['entry', 'Target', 'Traffic Demand'])
        for (i, j), demand in traffic_demand.items():
            writer.writerow([i, j, demand])
        
        # Écriture de l'ensemble S
        writer.writerow([])
        writer.writerow(['Subset S'])
        writer.writerow(S)

In [11]:
def create_instance(graph_type, n_nodes, capacity_range, cost_range, S_size, demand_range, filename):
    G = generate_graph(graph_type, n_nodes, p=0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)

    print(f"Instance générée et sauvegardée dans {filename}")
    
    

Exemple utilisation de géneration, attribution des capacités et coût et de la sauvegarde du graphe 

In [12]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 15  # Nombre de nœuds
    capacity_range = (1, 10)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_instance2.csv'  # Nom du fichier CSV

    create_instance(graph_type, n_nodes, capacity_range, cost_range, S_size, demand_range, filename)
    
    

Instance générée et sauvegardée dans graph_instance2.csv


Fonction d'une lecture d'une instance 

In [13]:
def read_instance(filename):
    """
    Lit une instance à partir d'un fichier CSV.
    Args:
        filename (string): nom du fichier csv
    Return:
        Graphe, sous-ensemble S, et demande de trafic  
    """
    G = nx.DiGraph()
    traffic_demand = {}
    S = []
    with open("graph/"+filename, mode='r') as file:
        reader = csv.reader(file)
        section = "arcs"
        for row in reader:
            if not row:
                continue
            if row[0] == "Source":
                continue
            if row[0] == "entry":
                section = "traffic"
            if row[0] == "Subset S":
                section = "subset"
                continue
            if section == "arcs":
                G.add_edge(int(row[0]), int(row[1]), capacity=float(row[2]), cost=float(row[3]))
            elif section =="traffic":
                traffic_demand[(int(row[0]), int(row[1]))] = int(row[2])
            elif section == "subset":
                S = list(map(int, row))
                break
    return G, S, traffic_demand 
    

Fonction pour créer et resoudre un PL pour une fonction objectif donnée 

In [14]:
def solve_linear_program(G, traffic_demand, objective):
    """
    Résout un programme linéaire associé à une fonction objective spécifiée.

    Args:
        G (networkx graph): graphe de l'instance 
        traffic_demand (_type_): valeur de la demande pour le sous ensemble S
        objective (_type_): fonction objective choisie
        
    return:
        solutions de routage et valeur optimale 
    """
    
    model = Model("Routing")
    
    #Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[i, j, u, v] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")
    
    #Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v, in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Constrainte de capacité 
    for u, v in G.edges:    
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )
    
    # Objectifs
    if objective == "cost":
        model.setObjective(
            sum(x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges),
            GRB.MINIMIZE
        )
    elif objective == "max_utlilization":
        utilisation = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilisation")
        for u, v in G.edges:
            model.addConstr(
                utilisation >=
                sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
                name=f"utilisation_{u}_{v}"
            )
        model.setObjective(utilisation, GRB.MINIMIZE)
    elif objective == "avg_utilisation":
        avg_utilisation = sum(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
            for u, v in G.edges
        ) / G.number_of_edges()
        model.setObjective(avg_utilisation, GRB.MINIMIZE)
    else:
        raise ValueError("Fonction objectif non reconnue")

    #Résolution
    model.optimize()
    
    # Débogage en cas d'infaisabilité
    if model.status == GRB.INFEASIBLE:
        print("Modèle infaisable. Analyse des conflits...")
        model.computeIIS()
        model.write("infeasible.ilp")
        print("Conflits enregistrés dans infeasible.ilp")
        return None
    
    #Extraction de la solution
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, model.ObjVal
    else:
        raise Exception("Pas de solution optimale trouvée")
    

Fonction de test de la résolution d'une instance de graphe

In [15]:
def test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename):
    """
    Teste le programme de résolution de PL sur un type de graphe

    Args:
        graph_type (string): type de graphe 
        n_nodes (int): nombre de node dans V
        S_size (int): nombre de node dans S
        demand_range (tuple): tuple de 2 int pour le range des valeurs de demande 
        capacity_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        cost_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        objective (string): nom de la fonction objective choisi (cost, max_utlilization, avg_utlilization)
    """
    G = generate_graph(graph_type, n_nodes, 0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)
    solution, optimal_value = solve_linear_program(G, traffic_demand, objective)
    print(f"Solution optimale pour {objective}:", solution)
    print(f"Valeur optimale pour {objective}: {optimal_value}")

Graphe de type Random

In [27]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 15  # Nombre de nœuds
    capacity_range = (10, 20)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_random.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename)
    
    

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 373 rows, 1460 columns and 4380 nonzeros
Model fingerprint: 0xbcbcf2da
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 20 rows and 0 columns
Presolve time: 0.02s
Presolved: 353 rows, 1460 columns, 4161 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.500000e+01   0.000000e+00      0s
     234    9.8133277e+02   0.000000e+00   0.000000e+00      0s

Solved in 234 iterations and 0.03 seconds (0.00 work units)
Optimal objective  9.813327696e+02
Solution optimale pour cost: {(13, 2, 3, 2): 1.0, (13, 2, 4, 3): 1.0, (13, 2, 13, 4): 1.0, (13, 0, 

Graphe de type complet

In [29]:
# Exemple d'utilisation pour un graphe complet
if __name__ == "__main__":
    # Paramètres pour le graphe complet
    graph_type = 'complete'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 8  # Nombre de nœuds
    capacity_range = (10, 20)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_complete.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 104 rows, 336 columns and 1008 nonzeros
Model fingerprint: 0xfa145377
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 98 rows, 336 columns, 924 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.600000e+01   0.000000e+00      0s
      38    2.1551595e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.155159547e+02
Solution optimale pour cost: {(7, 3, 7, 3): 1.0, (7, 0, 7, 0): 1.0, (3, 7, 3, 7): 1.0, (3, 0, 3, 0): 1.0,

Graphe de type cycle

In [35]:
# Exemple d'utilisation pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'cycle'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_cycle.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 36 columns and 108 nonzeros
Model fingerprint: 0x2a2bbebd
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 42 rows and 36 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1660903e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.166090321e+02
Solution optimale pour cost: {(1, 0, 1, 2): 1.0, (1, 0, 2, 3): 1.0, (1, 0, 3, 4): 1.0, (1, 0, 4, 5): 1.0, (1, 0, 5, 0): 1.0, (1, 2, 1, 2): 1.0, (0, 1, 0, 1): 1.0, (0, 2, 0, 1): 1.

Méthode de la somme pondérée

In [31]:
# Implémentation des méthodes d'obtention de solutions efficaces
def weighted_sum_method(G, traffic_demand, weights):
    """
    Implémente la méthode des sommes pondées pour le routage multiobjectif.
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param weights: Poids pour les objectifs (liste de trois poids).
    :return: Solution optimale et valeur des objectifs.
    """
    model = Model("WeightedSumRouting")

    # Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Objectifs individuels
    cost = sum(
        x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges
    )

    max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
    for u, v in G.edges:
        model.addConstr(
            max_utilization >=
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
            name=f"utilization_{u}_{v}"
        )

    avg_utilization = sum(
        sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
        for u, v in G.edges
    ) / G.number_of_edges()

    # Fonction objectif pondérée
    model.setObjective(weights[0] * cost + weights[1] * max_utilization + weights[2] * avg_utilization, GRB.MINIMIZE)

    # Résolution
    model.optimize()

    # Extraction des résultats
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, cost.getValue(), max_utilization.x, avg_utilization.getValue()
    else:
        raise Exception("Modèle non résolu (statut: {model.status})")

def test_weighted_sum(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename):
    """
    Teste le programme de résolution de PL sur un type de graphe

    Args:
        graph_type (string): type de graphe 
        n_nodes (int): nombre de node dans V
        S_size (int): nombre de node dans S
        demand_range (tuple): tuple de 2 int pour le range des valeurs de demande 
        capacity_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        cost_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        objective (string): nom de la fonction objective choisi (cost, max_utlilization, avg_utlilization)
    """
    G = generate_graph(graph_type, n_nodes, 0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)
    weights = [0.5, 0.3, 0.2]  # Pondérations des objectifs
    solution, cost, max_util, avg_util = weighted_sum_method(G, traffic_demand, weights)
    print("Solution optimale :", solution)
    print("Coût total :", cost)
    print("Utilisation maximale :", max_util)
    print("Utilisation moyenne :", avg_util)

Test des différents graphe pour la méthode de la somme pondérée

Graphe cycle

In [34]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'cycle'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_cycle_weighted_sum.csv'  # Nom du fichier CSV
    test_weighted_sum(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 37 columns and 150 nonzeros
Model fingerprint: 0xe5b4c485
Coefficient statistics:
  Matrix range     [8e-02, 2e+01]
  Objective range  [3e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 48 rows and 37 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0153975e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.015397463e+02
Solution optimale : {(2, 5, 2, 3): 1.0, (2, 5, 3, 4): 1.0, (2, 5, 4, 5): 1.0, (2, 0, 2, 3): 1.0, (2, 0, 3, 4): 1.0, (2, 0, 4, 5): 1.0, (2, 0, 5, 0): 1.0, (5, 2, 0, 1): 1.0, (5, 2,

Graphe complet 

In [36]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'complete'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_complete_weighted_sum.csv'  # Nom du fichier CSV
    test_weighted_sum(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 96 rows, 181 columns and 750 nonzeros
Model fingerprint: 0xe33f4f03
Coefficient statistics:
  Matrix range     [3e-02, 2e+01]
  Objective range  [3e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 90 rows, 181 columns, 690 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.600000e+01   0.000000e+00      0s
      15    7.5009751e+01   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.500975122e+01
Solution optimale : {(4, 0, 2, 0): 1.0, (4, 0, 4, 2): 1.0, (4, 3, 4, 3): 1.0, (0, 4, 0, 3): 1.0, (0, 4, 3, 

Graphe random

In [37]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_random_weighted_sum.csv'  # Nom du fichier CSV
    test_weighted_sum(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 62 rows, 79 columns and 325 nonzeros
Model fingerprint: 0xba567c9d
Coefficient statistics:
  Matrix range     [7e-02, 2e+01]
  Objective range  [3e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 27 rows and 31 columns
Presolve time: 0.02s
Presolved: 35 rows, 48 columns, 170 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5086426e+02   1.100000e+01   0.000000e+00      0s
      10    2.2926257e+02   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.04 seconds (0.00 work units)
Optimal objective  2.292625660e+02
Solution optimale : {(2, 5, 2, 5): 1.0, (2, 0, 2, 3): 1.0, (2, 0, 3, 4): 1.0, (2, 0, 4, 0): 1.0, (5, 2, 5, 

Epsilon contrainte 

In [33]:
def epsilon_constraint_method(G, traffic_demand, primary_objective, secondary_objective, epsilon):
    """
    Implémente la méthode des ε-contraintes pour le routage multiobjectif.
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param primary_objective: Objectif principal ('cost', 'max_utilization', 'avg_utilization').
    :param secondary_objective: Objectif secondaire pour les contraintes.
    :param epsilon: Valeur limite pour l'objectif secondaire.
    :return: Solution optimale et valeur des objectifs.
    """
    model = Model("EpsilonConstraintRouting")

    # Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Objectifs individuels
    cost = sum(
        x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges
    )

    max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
    for u, v in G.edges:
        model.addConstr(
            max_utilization >=
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
            name=f"utilization_{u}_{v}"
        )

    avg_utilization = sum(
        sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
        for u, v in G.edges
    ) / G.number_of_edges()

    # Contraintes sur l'objectif secondaire
    if secondary_objective == 'cost':
        model.addConstr(cost <= epsilon, name="epsilon_cost")
    elif secondary_objective == 'max_utilization':
        model.addConstr(max_utilization <= epsilon, name="epsilon_max_utilization")
    elif secondary_objective == 'avg_utilization':
        model.addConstr(avg_utilization <= epsilon, name="epsilon_avg_utilization")
    else:
        raise ValueError("Objectif secondaire non reconnu")

    # Objectif principal
    if primary_objective == 'cost':
        model.setObjective(cost, GRB.MINIMIZE)
    elif primary_objective == 'max_utilization':
        model.setObjective(max_utilization, GRB.MINIMIZE)
    elif primary_objective == 'avg_utilization':
        model.setObjective(avg_utilization, GRB.MINIMIZE)
    else:
        raise ValueError("Objectif principal non reconnu")

    # Résolution
    model.optimize()

    # Extraction des résultats
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, cost.getValue(), max_utilization.x, avg_utilization.getValue()
    else:
        raise Exception("Modèle non résolu (statut: {model.status})")

def test_epsilon_constraint(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename):
    """
    Teste le programme de résolution de PL sur un type de graphe

    Args:
        graph_type (string): type de graphe 
        n_nodes (int): nombre de node dans V
        S_size (int): nombre de node dans S
        demand_range (tuple): tuple de 2 int pour le range des valeurs de demande 
        capacity_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        cost_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        objective (string): nom de la fonction objective choisi (cost, max_utlilization, avg_utlilization)
    """
    G = generate_graph(graph_type, n_nodes, 0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)
    primary_objective = 'cost'
    secondary_objective = 'max_utilization'
    epsilon = 2.5
    solution, cost, max_util, avg_util = epsilon_constraint_method(G, traffic_demand, primary_objective, secondary_objective, epsilon)
    print("Solution optimale :", solution)
    print("Coût total :", cost)
    print("Utilisation maximale :", max_util)
    print("Utilisation moyenne :", avg_util)

Test des différents type de graphe pour epsilon methode

Graphe cycle 

In [38]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'cycle'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_cycle_epsilon_constraint.csv'  # Nom du fichier CSV
    test_epsilon_constraint(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 37 columns and 151 nonzeros
Model fingerprint: 0xd3b9606e
Coefficient statistics:
  Matrix range     [2e-02, 2e+01]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 49 rows and 37 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7144552e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.714455201e+02
Solution optimale : {(3, 5, 3, 4): 1.0, (3, 5, 4, 5): 1.0, (3, 4, 3, 4): 1.0, (5, 3, 0, 1): 1.0, (5, 3, 1, 2): 1.0, (5, 3, 2, 3): 1.0, (5, 3, 5, 0): 1.0, (5, 4, 0, 1): 1.0, (5, 4,

Graphe complete 

In [39]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'complete'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_complete_epsilon_constraint.csv'  # Nom du fichier CSV
    test_epsilon_constraint(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 97 rows, 181 columns and 751 nonzeros
Model fingerprint: 0x53770af7
Coefficient statistics:
  Matrix range     [8e-02, 2e+01]
  Objective range  [6e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 37 rows and 1 columns
Presolve time: 0.01s
Presolved: 60 rows, 180 columns, 480 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.300000e+01   0.000000e+00      0s
      17    2.6418991e+02   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.641899086e+02
Solution optimale : {(1, 3, 1, 3): 1.0, (1, 5, 1, 5): 1.0, (3, 1, 3, 1): 1.0, (3, 5, 3, 5): 1.0, (5, 1, 0,

Graphe random

In [55]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_random_epsilon_constraint.csv'  # Nom du fichier CSV
    test_epsilon_constraint(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 67 columns and 276 nonzeros
Model fingerprint: 0x5af61394
Coefficient statistics:
  Matrix range     [5e-02, 2e+01]
  Objective range  [4e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 59 rows and 67 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.2779085e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.277908501e+02
Solution optimale : {(1, 0, 1, 3): 1.0, (1, 0, 3, 0): 1.0, (1, 5, 1, 5): 1.0, (0, 1, 0, 3): 1.0, (0, 1, 3, 1): 1.0, (0, 5, 0, 3): 1.0, (0, 5, 1, 5): 1.0, (0, 5, 3, 1): 1.0, (5, 1,

Méthode lexicographique

In [58]:
def lexicographic_method(G, traffic_demand, objectives):
    """
    Implémente la méthode d'optimisation lexicographique.
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param objectives: Liste d'objectifs par ordre de priorité ('cost', 'max_utilization', 'avg_utilization').
    :return: Solution optimale et valeurs des objectifs.
    """
    model = Model("LexicographicRouting")

    # Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Objectifs individuels
    cost = sum(
        x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges
    )

    max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
    for u, v in G.edges:
        model.addConstr(
            max_utilization >=
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
            name=f"utilization_{u}_{v}"
        )

    avg_utilization = sum(
        sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
        for u, v in G.edges
    ) / G.number_of_edges()

    # Résolution lexicographique
    results = {}
    for objective in objectives:
        if objective == 'cost':
            model.setObjective(cost, GRB.MINIMIZE)
        elif objective == 'max_utilization':
            model.setObjective(max_utilization, GRB.MINIMIZE)
        elif objective == 'avg_utilization':
            model.setObjective(avg_utilization, GRB.MINIMIZE)
        else:
            raise ValueError("Objectif non reconnu")

        model.optimize()

        if model.status == GRB.OPTIMAL:
            results[objective] = model.objVal

            # Ajouter les contraintes pour figer l'objectif courant
            if objective == 'cost':
                model.addConstr(cost <= model.objVal, name="fix_cost")
            elif objective == 'max_utilization':
                model.addConstr(max_utilization <= model.objVal, name="fix_max_utilization")
            elif objective == 'avg_utilization':
                model.addConstr(avg_utilization <= model.objVal, name="fix_avg_utilization")
        else:
            raise Exception(f"Modèle non résolu pour l'objectif {objective} (statut: {model.status})")

    # Extraction des résultats finaux
    solution = {key: var.x for key, var in x.items() if var.x > 0}
    return solution, results

def test_lexicographique(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename):
    """
    Teste le programme de résolution de PL sur un type de graphe

    Args:
        graph_type (string): type de graphe 
        n_nodes (int): nombre de node dans V
        S_size (int): nombre de node dans S
        demand_range (tuple): tuple de 2 int pour le range des valeurs de demande 
        capacity_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        cost_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        objective (string): nom de la fonction objective choisi (cost, max_utlilization, avg_utlilization)
    """
    G = generate_graph(graph_type, n_nodes, 0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)
    primary_objective = 'cost'
    secondary_objective = 'max_utilization'
    epsilon = 2.5
    objectives = ['cost', 'max_utilization', 'avg_utilization']
    solution, results = lexicographic_method(G, traffic_demand, objectives)
    print("Solution optimale :", solution)
    print("Résultats lexicographiques :", results)

Test sur les différents types de graphe de la méthode lexicographique

Graphe cycle

In [130]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'cycle'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (20, 40)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (5, 10)  # Intervalle des demandes de trafic
    filename = 'graph_cycle_lexicographique.csv'  # Nom du fichier CSV
    test_lexicographique(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 37 columns and 150 nonzeros
Model fingerprint: 0x50bab6a2
Coefficient statistics:
  Matrix range     [2e-01, 1e+01]
  Objective range  [8e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 48 rows and 37 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.3607364e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.360736392e+02
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread

Graphe complete

In [145]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'complete'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (1, 10)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_complete_lexicographique.csv'  # Nom du fichier CSV
    test_epsilon_constraint(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 97 rows, 181 columns and 751 nonzeros
Model fingerprint: 0x3e86d923
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 37 rows and 1 columns
Presolve time: 0.01s
Presolved: 60 rows, 180 columns, 480 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.800000e+01   0.000000e+00      0s
      42    2.0197618e+02   0.000000e+00   0.000000e+00      0s

Solved in 42 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.019761797e+02
Solution optimale : {(3, 2, 1, 2): 0.16666666666666674, (3, 2, 3, 1): 0.16666666666666674, (3, 2, 3, 2): 0

Graphe random

In [222]:
# Exemple d'utilisation de la somme pondérée pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 12  # Nombre de nœuds
    capacity_range = (3, 8)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (4, 8)  # Intervalle des demandes de trafic
    filename = 'graph_random_lexicographique.csv'  # Nom du fichier CSV
    test_epsilon_constraint(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 155 rows, 247 columns and 1026 nonzeros
Model fingerprint: 0x07116d39
Coefficient statistics:
  Matrix range     [5e-01, 8e+00]
  Objective range  [4e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 58 rows and 10 columns
Presolve time: 0.03s
Presolved: 97 rows, 237 columns, 651 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.200000e+01   0.000000e+00      0s
      37    1.6870586e+02   0.000000e+00   0.000000e+00      0s

Solved in 37 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.687058639e+02
Solution optimale : {(3, 8, 2, 8): 0.375, (3, 8, 3, 2): 0.375, (3, 8, 3, 6): 0.625, (3, 8, 5, 8): 0.625

<h3> Routage incertain </h3>

Géneration des données

In [ ]:
# Gestion de l'incertitude dans les problèmes de routage
def generate_uncertainty_sets(G, traffic_demand, demand_range, capacity_range):
    """
    Génère des ensembles d'incertitude pour les vecteurs de trafic (UT) et de coûts (UW).
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param demand_range: Intervalle des demandes de trafic pour UT.
    :param capacity_range: Intervalle des capacités pour UW.
    :return: Ensembles UT et UW.
    """
    # Générer UT de type 'modèle hose'
    UT = {}
    for (i, j), demand in traffic_demand.items():
        UT[(i, j)] = {
            "min": max(0, demand - demand_range),
            "max": demand + demand_range
        }

    # Générer UW comme bornes inférieures et supérieures sur les coûts
    UW = {}
    for u, v in G.edges:
        UW[(u, v)] = {
            "min": max(0, G[u][v]['cost'] - capacity_range),
            "max": G[u][v]['cost'] + capacity_range
        }

    return UT, UW

def robust_optimization(G, UT, UW, objective):
    """
    Résout le problème de routage en tenant compte de l'incertitude via optimisation robuste.
    :param G: Graphe orienté.
    :param UT: Ensemble d'incertitude pour le trafic.
    :param UW: Ensemble d'incertitude pour les coûts.
    :param objective: Fonction objectif ('cost', 'max_utilization', 'avg_utilization').
    :return: Solution robuste et valeur optimale.
    """
    model = Model("RobustRouting")

    # Variables de décision
    x = {}
    for u, v in G.edges:
        for (i, j), bounds in UT.items():
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), bounds in UT.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(bounds['max'] * x[(i, j, u, v)] for (i, j), bounds in UT.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Fonction objectif robuste
    if objective == 'cost':
        cost = sum(
            UW[(u, v)]['max'] * x[(i, j, u, v)] for u, v in G.edges for (i, j), bounds in UT.items()
        )
        model.setObjective(cost, GRB.MINIMIZE)
    elif objective == 'max_utilization':
        max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
        for u, v in G.edges:
            model.addConstr(
                max_utilization >=
                sum(bounds['max'] * x[(i, j, u, v)] for (i, j), bounds in UT.items()) / G[u][v]['capacity'],
                name=f"utilization_{u}_{v}"
            )
        model.setObjective(max_utilization, GRB.MINIMIZE)
    elif objective == 'avg_utilization':
        avg_utilization = sum(
            sum(bounds['max'] * x[(i, j, u, v)] for (i, j), bounds in UT.items()) / G[u][v]['capacity']
            for u, v in G.edges
        ) / G.number_of_edges()
        model.setObjective(avg_utilization, GRB.MINIMIZE)
    else:
        raise ValueError("Objectif non reconnu")

    # Résolution
    model.optimize()

    # Extraction des résultats
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, model.objVal
    else:
        raise Exception("Modèle non résolu (statut: {model.status})")

# Exemple d'utilisation
def test_robust_optimization():
    # Créer un graphe simple
    G = nx.DiGraph()
    G.add_edge(0, 1, capacity=5, cost=2)
    G.add_edge(1, 2, capacity=5, cost=3)
    G.add_edge(0, 2, capacity=1, cost=1)

    traffic_demand = {(0, 2): 3, (0, 1): 2}
    demand_range = 1
    capacity_range = 0.5

    # Générer les ensembles d'incertitude
    UT, UW = generate_uncertainty_sets(G, traffic_demand, demand_range, capacity_range)

    # Résoudre le problème robuste
    solution, obj_val = robust_optimization(G, UT, UW, 'cost')
    print("Solution robuste :", solution)
    print("Valeur optimale :", obj_val)

# Lancer le test
test_robust_optimization()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 6 columns and 18 nonzeros
Model fingerprint: 0x01ad3673
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 6 rows and 3 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model


Exception: Modèle non résolu (statut: {model.status})